# SOI1010 Machine Learning II - Assignment #3
Due: December 8, 2023 11:59 pm

You know the drill. The submission should include a code (both link to the colab and .py format) and a report that has answers to the questions and results. Use PyTorch (or TensorFlow/JAX). Also, minimize the use of numpy. There will be mark deductions if numpy is used when PyTorch is preferable or should be used. Marks will be deducted if the submission does not include the requested files. DO NOT use other libraries, such as scikit-learn/sklearn, to use a model you are supposed to implement. Using sklearn or any other third library or already built-in functions that you are asked to implement will result in 0 mark. Also, if an assignment asks you to implement some model, that means you shouldn’t use the built-in implementation from any library for that model in the first place.


NEW guidelines:
1. Do NOT copy codes from references/online resources, especially from DACON. If you get caught copying/combining parts of the codes that are being shared online, you will get 0.
2. Do not use test data (from test.csv) for training your model. Test data should only be used for test. If the results are not reproducible, you will get 0.
3. Do not use external data. Stick with what you are provided with from DACON compe- tition.
4. Do not use a pre-trained model. You should train a model from scratch.
5. In other words, use only train data (from train.csv) for training your model.

### DACON competition: Sign Language Recognition

a) The competition/dataset can be found [here](https://dacon.io/en/competitions/official/235896/overview/description).

In [7]:
!kaggle datasets download -d nahyunpark/korean-sign-languageksl-numbers

100%|██████████████████████████████████████| 2.08G/2.08G [10:50<00:00, 3.60MB/s]
100%|██████████████████████████████████████| 2.08G/2.08G [10:50<00:00, 3.43MB/s]


b) Your task is to achieve as high performance as possible (without cheating, of course). You can use any model you want. But, I suggest that you start with MLP. I strongly encourage you to research and study advanced neural network models and implement them to get better performance.


In [14]:
import os
import torch
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

In [15]:
#하이퍼 파라미터 튜닝

CFG = {
    'IMG_SIZE':128, #이미지 사이즈
    'EPOCHS':50, #에포크
    'LEARNING_RATE':2e-2, #학습률
    'BATCH_SIZE':12, #배치사이즈
    'SEED':41, #시드
}

In [16]:
# Seed 고정
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [17]:
import pandas as pd
label_df = pd.read_csv('data/train.csv')
label_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.csv'